In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import dispatch_model
from accelerate.utils import infer_auto_device_map
import accelerate

#model/thoth/thoth-1b2-v3
#model/Qwen2.5-Coder-3B-Instruct

model_name = "model/Qwen2.5-Coder-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.83s/it]


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
write a quick sort algorithm.<|im_end|>
<|im_start|>assistant



In [3]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   4934,    264,   3974,
           3378,  12111,     13, 151645,    198, 151644,  77091,    198]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}


In [4]:

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


In [5]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Certainly! Quick Sort is a popular and efficient sorting algorithm that uses a divide-and-conquer approach. Below is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]  # Choosing the middle element as the pivot
        left = [x for x in arr if x < pivot]  # Elements less than the pivot
        middle = [x for x in arr if x == pivot]  # Elements equal to the pivot
        right = [x for x in arr if x > pivot]  # Elements greater than the pivot
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
array = [3, 6, 8, 10, 1, 2, 1]
sorted_array = quick_sort(array)
print(sorted_array)
```

### Explanation:
1. **Base Case**: If the array has 0 or 1 element, it is already sorted, so we return it as is.
2. **Pivot Selection**: We choose the middle element of the array as the pivot.
3. **Partitioning**:
   - We create three lists: `left` for 